
GROUP MEMBERS:

Asad Karim -- 226063

Shweta Pandey -- 225964

In [0]:
import tensorflow as tf
import numpy as np

In [2]:
!python prepare_data2.py shakespeare.txt skp \\n\\n+ -m 200

2020-06-02 06:42:08.165812: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 7222 sequences...
Longest sequence is 3082 characters. If this seems unreasonable, consider using the maxlen argument!
Removing sequences longer than 200 characters...
5666 sequences remaining.
Longest remaining sequence has length 199.
Removing length-0 sequences...
5666 sequences remaining.
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
S

In [3]:
from prepare_data2 import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'\n': 3, '-': 4, ':': 5, '?': 6, 'S': 7, 'I': 8, 'i': 9, 'Z': 10, 'b': 11, 'n': 12, 'P': 13, 'V': 14, 'C': 15, 'q': 16, '&': 17, 'Q': 18, 'd': 19, 'R': 20, 'o': 21, '$': 22, 'f': 23, 'J': 24, 'L': 25, 'j': 26, 'E': 27, 'h': 28, 'p': 29, 'm': 30, '3': 31, 'l': 32, 'K': 33, 'W': 34, 'H': 35, ';': 36, 'D': 37, 's': 38, 'G': 39, 'N': 40, 'e': 41, 'B': 42, 'r': 43, 'z': 44, ',': 45, 'a': 46, '.': 47, 'M': 48, '!': 49, 'O': 50, 'v': 51, "'": 52, 'U': 53, 'w': 54, 'A': 55, 'X': 56, ' ': 57, 'g': 58, 'u': 59, 'F': 60, 'c': 61, 't': 62, 'Y': 63, 'T': 64, 'x': 65, 'k': 66, 'y': 67, '<PAD>': 0, '<S>': 1, '</S>': 2}
68


In [0]:
SEQ_LENGTH = 200
train_data = data.shuffle(30000).padded_batch(128,
padded_shapes=[SEQ_LENGTH],drop_remainder=True).repeat()


In [0]:
def split_input_target(chunk):
  input_text = chunk[:,:-1]
  target_text = chunk[:,1:]
  return input_text, target_text

In [0]:
def split_input_target2(chunk):
  input_text = chunk[:,:-1]
  target_text = chunk[:,1:]
  input_text = tf.one_hot(input_text, depth = vocab_size)
  target_text = tf.one_hot(target_text, depth = vocab_size)
  return input_text, target_text

In [0]:
train_data = train_data.map(split_input_target)

In [92]:
train_data.take(1)

<TakeDataset shapes: ((128, 199), (128, 199)), types: (tf.int32, tf.int32)>

In [0]:
train_data2 = train_data.map(split_input_target2)

In [112]:
for input_text, target_text in train_data.take(1):
  print(target_text)
  as_chars = [ind_to_ch[ind] for ind in input_text[3].numpy()]
  print("Input data: "+ "".join(as_chars), "\n")
  as_chars = [ind_to_ch[ind] for ind in target_text[3].numpy()]
  print("Target data: "+ "".join(as_chars), "\n")


tf.Tensor(
[[33  8 40 ...  0  0  0]
 [64 20 55 ...  0  0  0]
 [37 53 33 ...  0  0  0]
 ...
 [ 7 27 42 ...  0  0  0]
 [35 27 40 ...  0  0  0]
 [60 20 50 ...  0  0  0]], shape=(128, 199), dtype=int32)
Input data: <S>LEONTES:
Go on, go on
Thou canst not speak too much; I have deserved
All tongues to talk their bitterest.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD> 

Target data: LEONTES:
Go on, go on
Thou canst not speak too much; I have deserved
All tongues to talk their bitterest.</S><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><P

In [84]:
for input_text2, target_text2 in train_data2.take(1):
  print(input_text2)
  as_chars = [ind_to_ch[ind] for ind in input_text2[3].numpy()]
  print("Input data: "+ "".join(as_chars), "\n")
  as_chars = [ind_to_ch[ind] for ind in target_text2[3].numpy()]
  print("Target data: "+ "".join(as_chars), "\n")

tf.Tensor(
[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]

TypeError: ignored

In [0]:
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024
rnn_units = 512
batch_size = 128

In [0]:

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.SimpleRNN(rnn_units,
                        return_sequences=True,
                        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=batch_size)

In [125]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (128, None, 256)          17408     
_________________________________________________________________
simple_rnn (SimpleRNN)       (128, None, 512)          393728    
_________________________________________________________________
dense_2 (Dense)              (128, None, 68)           34884     
Total params: 446,020
Trainable params: 446,020
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
EPOCHS = 10

In [130]:
model.fit(train_data,epochs=EPOCHS,steps_per_epoch= 10)

Epoch 1/10


TypeError: ignored

In [0]:
def loss(labels, logits):
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)

In [141]:
batch_loss = 0
for batch, target in train_data.take(1):
  print(batch.shape)
  print(batch)
  #oh_batch = tf.one_hot(batch, depth = vocab_size)
  #print(oh_batch.shape)
  batch_predictions = model(batch)
  print(batch_predictions.shape)
  mask = [tf.math.count_nonzero(seq) for seq in batch]
  print(len(mask))
  no_of_elements = tf.reduce_sum(mask)
  print(no_of_elements)

  seq_mask = tf.sequence_mask(mask, SEQ_LENGTH, dtype = tf.dtypes.float32)
  print(seq_mask.shape)
  seq_mask = seq_mask[:, :-1]
  print(seq_mask.shape)
  print(seq_mask)
  for t in range(1,128):
    sampled_indices = tf.random.categorical(batch_predictions[t], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
    reduced = sampled_indices * seq_mask[t]
    batch_loss += loss(target[t], reduced)
  avg_batch_loss = batch_loss/no_of_elements


(128, 199)
tf.Tensor(
[[ 1 33  8 ...  0  0  0]
 [ 1 15 55 ...  0  0  0]
 [ 1 25 27 ...  0  0  0]
 ...
 [ 1 18 53 ...  0  0  0]
 [ 1 15 55 ...  0  0  0]
 [ 1 24 50 ...  0  0  0]], shape=(128, 199), dtype=int32)
(128, 199, 68)
128
tf.Tensor(10317, shape=(), dtype=int64)
(128, 200)
(128, 199)
tf.Tensor(
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]], shape=(128, 199), dtype=float32)


ValueError: ignored

The code didn't work, I tried running the model with SimpleRNN without a embedding layer, but value parameter passed didn't work. with error 'Value passed to parameter 'labels' has DataType float32 not in list of allowed values: int32, int64'